In [1764]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
import time

name_file_data_report = 'Общая таблица слушателей ЦОПП 1-ПО.xlsx' 
path_to_end_folder_report = 'data'

df_po = pd.read_excel(name_file_data_report,sheet_name='ПО',dtype={'Гражданство_получателя_код_страны_по_ОКСМ':str})

#Создаем шрифт которым будем выделять названия таблиц
font_name_table = Font(name='Arial Black',size=15,italic=True)

1.3,
2.1.1,
2.1.2,
2.1.3,
2.2 Добавить категории слушателей для 1-ПО
2.3
2.4
2.5
2.6


In [1765]:
# Создаем файл excel
wb = openpyxl.Workbook()
# Создаем листы 
wb.create_sheet(title='Раздел 1.3',index=0)
wb.create_sheet(title='Раздел 2.1.1',index=1)
wb.create_sheet(title='Раздел 2.1.2',index=2)
wb.create_sheet(title='Раздел 2.1.3',index=3)
wb.create_sheet(title='Раздел 2.4',index=4)
wb.create_sheet(title='Раздел 2.5',index=5)
wb.create_sheet(title='Раздел 2.6',index=6)



<Worksheet "Раздел 2.6">

In [1766]:
df_po_1_3_base = df_po.copy()
df_po_1_3_base['for_counting'] = 1

In [1767]:
#Считаем лист 1.3
# Считаем количество реализованных программ
# группируем. Так как нам нужны текстовые данные то применяем создаем строку с помощью join
po_quantity_program_on_type_provisional =df_po_1_3_base.groupby('Наименование_программы_профессионального_обучения').agg({'Программа_профессионального_обучения_направление_подготовки':lambda x:';'.join(x)})
# Применяем к полученной серии функцию разделения по запятой. Предполо
po_quantity_program_on_type_provisional['Программа_профессионального_обучения_направление_подготовки']=po_quantity_program_on_type_provisional['Программа_профессионального_обучения_направление_подготовки'].apply(lambda x: x.split(';')[0])
df_po_1_3=po_quantity_program_on_type_provisional['Программа_профессионального_обучения_направление_подготовки'].value_counts().to_frame()

In [1768]:
# переименовываем колонку с
df_po_1_3.rename(columns={'Программа_профессионального_обучения_направление_подготовки':'Число реализованных программ'},inplace=True)

In [1769]:
# Сортирируем индекс чтобы было легче добавлять столбцы
df_po_1_3.sort_index(ascending=False,inplace=True)

In [1770]:
# Считаем количество обученных по каждой программе
df_po_1_3_quantity_students = df_po_1_3_base.groupby(['Программа_профессионального_обучения_направление_подготовки']).agg({'for_counting':'sum'})

In [1771]:
df_po_1_3_quantity_students.sort_index(ascending=False,inplace=True)

In [1772]:
# Соединяем 2 датафрейма
df_po_1_3['Всего обучено'] = df_po_1_3_quantity_students


In [1773]:
df_po_1_3

,Число реализованных программ,Всего обучено
"Программа профессиональной подготовки по профессии рабочего, должности служащего",1,4
"Программа повышения квалификации рабочих, служащих",1,2
"Программа переподготовки рабочих, служащих",1,6


In [1774]:
df_po_1_3_base.shape

(12, 63)

In [1775]:
# Считаем количество программ с сетевой формой
#Создаем датафрейм с теми данными о сетевой форме
df_po_1_3_network_base = df_po_1_3_base[df_po_1_3_base['Использование_сетевой_формы_обучения']=='Сетевая форма']


In [1776]:
# Считаем количество реализованных программ
# группируем. Так как нам нужны текстовые данные то применяем создаем строку с помощью join
po_quantity_program_on_type_network =df_po_1_3_network_base.groupby('Наименование_программы_профессионального_обучения').agg({'Программа_профессионального_обучения_направление_подготовки':lambda x:';'.join(x)})
# Применяем к полученной серии функцию разделения по запятой. Предполо
po_quantity_program_on_type_network['Программа_профессионального_обучения_направление_подготовки']=po_quantity_program_on_type_network['Программа_профессионального_обучения_направление_подготовки'].apply(lambda x: x.split(';')[0])
df_po_1_3_network=po_quantity_program_on_type_network['Программа_профессионального_обучения_направление_подготовки'].value_counts().to_frame()

In [1777]:
df_po_1_3_network.rename(columns={'Программа_профессионального_обучения_направление_подготовки':'Число реализованных программ'},inplace=True)

In [1778]:
df_po_1_3_network.sort_index(ascending=False,inplace=True)

In [1779]:
df_po_1_3['Число программ с сетевой формой'] = df_po_1_3_network

In [1780]:
df_po_1_3

,Число реализованных программ,Всего обучено,Число программ с сетевой формой
"Программа профессиональной подготовки по профессии рабочего, должности служащего",1,4,NaN
"Программа повышения квалификации рабочих, служащих",1,2,1.0
"Программа переподготовки рабочих, служащих",1,6,1.0


In [1781]:
# Считаем число слушателей на сетевых программах
df_po_1_3['Численность слушателей сетевых программ']=df_po_1_3_network_base.groupby('Программа_профессионального_обучения_направление_подготовки').agg({'for_counting':'sum'})

In [1782]:
df_po_1_3

,Число реализованных программ,Всего обучено,Число программ с сетевой формой,Численность слушателей сетевых программ
"Программа профессиональной подготовки по профессии рабочего, должности служащего",1,4,NaN,NaN
"Программа повышения квалификации рабочих, служащих",1,2,1.0,2.0
"Программа переподготовки рабочих, служащих",1,6,1.0,6.0


In [1783]:
#Считаем электронное обучение
df_po_1_3_distant_ao = df_po_1_3_base[(df_po_1_3_base['Использование_ЭО'] != 'Без применения ЭО')]

In [1784]:
po_group_quantity_distant_ao = df_po_1_3_distant_ao.groupby('Наименование_программы_профессионального_обучения').agg({'Программа_профессионального_обучения_направление_подготовки':lambda x:';'.join(x)})

# Применяем к полученной серии функцию разделения по запятой. 
po_group_quantity_distant_ao['Программа_профессионального_обучения_направление_подготовки']=po_group_quantity_distant_ao['Программа_профессионального_обучения_направление_подготовки'].apply(lambda x: x.split(';')[0])
df_po_1_3['Число программ реализуемых с помощью ЭО'] = po_group_quantity_distant_ao['Программа_профессионального_обучения_направление_подготовки'].value_counts().to_frame()

In [1785]:
#Считаем количество слушателей
df_po_1_3['Численность слушателей обученных с применением ЭО'] = df_po_1_3_distant_ao.groupby('Программа_профессионального_обучения_направление_подготовки').agg({'for_counting':'sum'})

In [1786]:
df_po_1_3

,Число реализованных программ,Всего обучено,Число программ с сетевой формой,Численность слушателей сетевых программ,Число программ реализуемых с помощью ЭО,Численность слушателей обученных с применением ЭО
"Программа профессиональной подготовки по профессии рабочего, должности служащего",1,4,NaN,NaN,1.0,3.0
"Программа повышения квалификации рабочих, служащих",1,2,1.0,2.0,NaN,NaN
"Программа переподготовки рабочих, служащих",1,6,1.0,6.0,1.0,6.0


In [1787]:
#Считаем пользователей с исключительно ЭО
df_po_1_3_distant_only_ao = df_po_1_3_base[df_po_1_3_base['Использование_ЭО'] =='Исключительно с ЭО']

In [1788]:
df_po_1_3['Численность слушателей обученных только с ЭО '] = df_po_1_3_distant_only_ao.groupby('Программа_профессионального_обучения_направление_подготовки').agg({'for_counting':'sum'})

In [1789]:
df_po_1_3

,Число реализованных программ,Всего обучено,Число программ с сетевой формой,Численность слушателей сетевых программ,Число программ реализуемых с помощью ЭО,Численность слушателей обученных с применением ЭО,Численность слушателей обученных только с ЭО
"Программа профессиональной подготовки по профессии рабочего, должности служащего",1,4,NaN,NaN,1.0,3.0,2.0
"Программа повышения квалификации рабочих, служащих",1,2,1.0,2.0,NaN,NaN,NaN
"Программа переподготовки рабочих, служащих",1,6,1.0,6.0,1.0,6.0,NaN


In [1790]:
df_po_1_3_distant_dot= df_po_1_3_base[df_po_1_3_base['Использование_ДОТ'] !='Без применения ДОТ']

In [1791]:

po_group_quantity_distant_dot = df_po_1_3_distant_dot.groupby('Наименование_программы_профессионального_обучения').agg({'Программа_профессионального_обучения_направление_подготовки':lambda x:';'.join(x)})

# Применяем к полученной серии функцию разделения по запятой. 
po_group_quantity_distant_dot['Программа_профессионального_обучения_направление_подготовки']=po_group_quantity_distant_dot['Программа_профессионального_обучения_направление_подготовки'].apply(lambda x: x.split(';')[0])
df_po_1_3['Число программ реализуемых с помощью ДОТ'] = po_group_quantity_distant_dot['Программа_профессионального_обучения_направление_подготовки'].value_counts().to_frame()

In [1792]:
df_po_1_3['Численность слушателей обученных с применением ДОТ'] = df_po_1_3_distant_dot.groupby(['Программа_профессионального_обучения_направление_подготовки']).agg({'for_counting':'sum'})

In [1793]:
# Считаем слушателей обученных только с ДОТ
df_po_1_3_distant_dot_only = df_po_1_3_base[df_po_1_3_base['Использование_ДОТ'] =='Исключительно с ДОТ']

In [1794]:
df_po_1_3_distant_dot_only.shape

(5, 63)

In [1795]:
df_po_1_3['Численность слушателей обученных только с  ДОТ'] = df_po_1_3_distant_dot_only.groupby(['Программа_профессионального_обучения_направление_подготовки']).agg({'for_counting':'sum'})

In [1796]:
df_po_1_3.index.name ='Вид образовательных программ'

In [1797]:
#Записываем датафрейм на лист
wb['Раздел 1.3'][f'A1'] = 'Сведения о образовательных программах,реализуемых организацией'
wb['Раздел 1.3'][f'A1'].font = font_name_table

for r in dataframe_to_rows(df_po_1_3,index =True,header=True):
    if len(r) != 1:
        wb['Раздел 1.3'].append(r)
wb['Раздел 1.3'][f'A2'] = 'Наименование образовательных программ'
wb['Раздел 1.3'].column_dimensions['A'].width = 60
wb['Раздел 1.3'].column_dimensions['B'].width = 20
wb['Раздел 1.3'].column_dimensions['D'].width = 20
wb['Раздел 1.3'].column_dimensions['E'].width = 20
wb['Раздел 1.3'].column_dimensions['F'].width = 20
wb['Раздел 1.3'].column_dimensions['G'].width = 20
wb['Раздел 1.3'].column_dimensions['H'].width = 20
wb['Раздел 1.3'].column_dimensions['I'].width = 20
wb['Раздел 1.3'].column_dimensions['J'].width = 20
wb['Раздел 1.3'].column_dimensions['K'].width = 20


wb['Раздел 1.3']['B2'].alignment = Alignment(wrap_text=True)
wb['Раздел 1.3']['C2'].alignment = Alignment(wrap_text=True)
wb['Раздел 1.3']['D2'].alignment = Alignment(wrap_text=True)
wb['Раздел 1.3']['E2'].alignment = Alignment(wrap_text=True)
wb['Раздел 1.3']['F2'].alignment = Alignment(wrap_text=True)
wb['Раздел 1.3']['G2'].alignment = Alignment(wrap_text=True)
wb['Раздел 1.3']['H2'].alignment = Alignment(wrap_text=True)
wb['Раздел 1.3']['I2'].alignment = Alignment(wrap_text=True)
wb['Раздел 1.3']['J2'].alignment = Alignment(wrap_text=True)
wb['Раздел 1.3']['K2'].alignment = Alignment(wrap_text=True)



In [1798]:
# Создаем раздел 2.1.1
df_po_2_1_1_base = df_po.copy()
df_po_2_1_1_base['for_counting'] = 1

In [1799]:
# Отбираем слушателей обученных по программам проф подготовки по профессиям рабочих,должностям служащих
df_po_2_1_1_base = df_po_2_1_1_base[df_po_2_1_1_base['Программа_профессионального_обучения_направление_подготовки'] == 'Программа профессиональной подготовки по профессии рабочего, должности служащего']

In [1800]:
# Создаем сводную таблицу для колонок 6,7,8,9
df_po_2_1_1_6_9 = pd.pivot_table(df_po_2_1_1_base,index=['Код_профессии_при_наличии'],
                                columns=['Источник_финансирования_обучения'],
                                values=['for_counting'],
                                aggfunc='sum')
df_po_2_1_1_6_9.columns = df_po_2_1_1_6_9.columns.droplevel()

In [1801]:
df_po_2_1_1_6_9['Всего по источникам финансирования'] = df_po_2_1_1_6_9.sum(axis=1)

In [1802]:
#Создаем сводную таблицу для колонок 13,14,15
df_po_2_1_1_13_15 = pd.pivot_table(df_po_2_1_1_base,index=['Код_профессии_при_наличии'],
                                columns=['Форма_обучения'],
                                values=['for_counting'],
                                aggfunc='sum')
df_po_2_1_1_13_15.columns = df_po_2_1_1_13_15.columns.droplevel()

In [1803]:
df_po_2_1_1_13_15['Всего по форме обучения'] = df_po_2_1_1_13_15.sum(axis=1)

In [1804]:
df_po_2_1_1 = pd.concat([df_po_2_1_1_6_9,df_po_2_1_1_13_15],axis=1)

In [1805]:
# Считаем по категориям финансирования
df_po_2_1_1_cat_finance = pd.pivot_table(df_po_2_1_1_base,index=['Код_профессии_при_наличии'],
                                        columns=['Источник_финансирования_индикаторы_физ_лицо_юр_лицо_бюдж_ассигнования_собственные_средства_ЦОПП'],
                                        values=['for_counting'],
                                        aggfunc='sum')
df_po_2_1_1_cat_finance.columns = df_po_2_1_1_cat_finance.columns.droplevel()

In [1806]:
df_po_2_1_1 = pd.concat([df_po_2_1_1,df_po_2_1_1_cat_finance],axis=1)

In [1807]:
df_po_2_1_1

,Местный бюджет,Платное обучение,Федеральный бюджет,Всего по источникам финансирования,Заочная,Очная,Очно-заочная (вечерняя),Всего по форме обучения,бюджетные ассигнования,иное физ. Лицо,обучаемый (физ. Лицо),собственные средства ЦОПП
Код_профессии_при_наличии,,,,,,,,,,,,
17,NaN,1.0,1.0,2.0,1.0,1.0,NaN,2.0,NaN,1.0,NaN,1.0
45,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN
54,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN


In [1808]:
#Считаем женщин

df_po_2_1_1_women= pd.pivot_table(df_po_2_1_1_base,index=['Код_профессии_при_наличии'],
                                 columns=['Пол_получателя'],
                                 values=['for_counting'],
                                 aggfunc='sum')
df_po_2_1_1_women.columns = df_po_2_1_1_women.columns.droplevel()

In [1809]:
df_po_2_1_1 = pd.concat([df_po_2_1_1,df_po_2_1_1_women],axis=1)

In [1810]:
df_po_2_1_1

,Местный бюджет,Платное обучение,Федеральный бюджет,Всего по источникам финансирования,Заочная,Очная,Очно-заочная (вечерняя),Всего по форме обучения,бюджетные ассигнования,иное физ. Лицо,обучаемый (физ. Лицо),собственные средства ЦОПП,Жен,Муж
Код_профессии_при_наличии,,,,,,,,,,,,,,
17,NaN,1.0,1.0,2.0,1.0,1.0,NaN,2.0,NaN,1.0,NaN,1.0,2.0,NaN
45,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0
54,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN


In [1811]:
# Считаем тех кто прошел по индвидуальным учебным планам


df_po_2_1_1_ind_up = pd.pivot_table(df_po_2_1_1_base,index=['Код_профессии_при_наличии'],
                                 columns=['прошли_ускоренное_обучение_по_индивидуаль-ным_учебным_планам'],
                                 values=['for_counting'],
                                 aggfunc='sum')
df_po_2_1_1_ind_up.columns = df_po_2_1_1_ind_up.columns.droplevel()

In [1812]:
df_po_2_1_1 = pd.concat([df_po_2_1_1,df_po_2_1_1_ind_up],axis=1)

In [1813]:
# Создаем проверку
df_po_2_1_1['temp'] = df_po_2_1_1['Всего по источникам финансирования'] == df_po_2_1_1['Всего по форме обучения']

In [1814]:
df_po_2_1_1['Совпадение сумм столбцов 6-9 и 13-15'] = df_po_2_1_1['temp'].apply(lambda x:'СОВПАДАЕТ' if x is True else 'НЕ СОВПАДАЕТ!!!')
df_po_2_1_1.drop(columns='temp',inplace=True)
df_po_2_1_1.index.name = 'Код профессии при наличии'

In [1815]:
#Записываем датафрейм на лист
wb['Раздел 2.1.1'][f'A1'] = 'Распределение слушателей,обученных по программам профессиональной подготовки по професииям рабочих,должностям служащих'
wb['Раздел 2.1.1'][f'A1'].font = font_name_table

for r in dataframe_to_rows(df_po_2_1_1,index =True,header=True):
    if len(r) != 1:
        wb['Раздел 2.1.1'].append(r)
wb['Раздел 2.1.1'][f'A2'] = 'В таблице отображаются ТОЛЬКО ТЕ показатели которые присутствуют в ИСХОДНОЙ таблице!!!!'
wb['Раздел 2.1.1'][f'A2'].font = font_name_table
wb['Раздел 2.1.1']['A2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.1'][f'A3'] = 'Код профессии при наличии'
wb['Раздел 2.1.1'].column_dimensions['A'].width = 50
wb['Раздел 2.1.1'].column_dimensions['C'].width = 15
wb['Раздел 2.1.1'].column_dimensions['D'].width = 15





wb['Раздел 2.1.1']['B2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.1']['C2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.1']['D2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.1']['E2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.1']['F2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.1']['G2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.1']['H2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.1']['I2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.1']['J2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.1']['K2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.1']['L2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.1']['M2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.1']['Q2'].alignment = Alignment(wrap_text=True)


In [1816]:
# Считаем раздел 2.1.2
df_po_2_1_2_base = df_po.copy()
df_po_2_1_2_base['for_counting'] = 1

In [1817]:
# Отбираем слушателей обученных по программам переподготовки рабочих, служащих
df_po_2_1_2_base = df_po_2_1_2_base[df_po_2_1_2_base['Программа_профессионального_обучения_направление_подготовки'] == 'Программа переподготовки рабочих, служащих']

# Создаем сводную таблицу для колонок 6,7,8,9
df_po_2_1_2_6_9 = pd.pivot_table(df_po_2_1_2_base,index=['Код_профессии_при_наличии'],
                                columns=['Источник_финансирования_обучения'],
                                values=['for_counting'],
                                aggfunc='sum')
df_po_2_1_2_6_9.columns = df_po_2_1_2_6_9.columns.droplevel()

df_po_2_1_2_6_9['Всего по источникам финансирования'] = df_po_2_1_2_6_9.sum(axis=1)

#Создаем сводную таблицу для колонок 13,14,15
df_po_2_1_2_13_15 = pd.pivot_table(df_po_2_1_2_base,index=['Код_профессии_при_наличии'],
                                columns=['Форма_обучения'],
                                values=['for_counting'],
                                aggfunc='sum')
df_po_2_1_2_13_15.columns = df_po_2_1_2_13_15.columns.droplevel()

df_po_2_1_2_13_15['Всего по форме обучения'] = df_po_2_1_2_13_15.sum(axis=1)

df_po_2_1_2 = pd.concat([df_po_2_1_2_6_9,df_po_2_1_2_13_15],axis=1)

In [1818]:
# Считаем по категориям финансирования
df_po_2_1_2_cat_finance = pd.pivot_table(df_po_2_1_2_base,index=['Код_профессии_при_наличии'],
                                        columns=['Источник_финансирования_индикаторы_физ_лицо_юр_лицо_бюдж_ассигнования_собственные_средства_ЦОПП'],
                                        values=['for_counting'],
                                        aggfunc='sum')
df_po_2_1_2_cat_finance.columns = df_po_2_1_2_cat_finance.columns.droplevel()

df_po_2_1_2 = pd.concat([df_po_2_1_2,df_po_2_1_2_cat_finance],axis=1)




#Считаем женщин

df_po_2_1_2_women= pd.pivot_table(df_po_2_1_2_base,index=['Код_профессии_при_наличии'],
                                 columns=['Пол_получателя'],
                                 values=['for_counting'],
                                 aggfunc='sum')
df_po_2_1_2_women.columns = df_po_2_1_2_women.columns.droplevel()

df_po_2_1_2 = pd.concat([df_po_2_1_2,df_po_2_1_2_women],axis=1)



# Считаем тех кто прошел по индвидуальным учебным планам


df_po_2_1_2_ind_up = pd.pivot_table(df_po_2_1_2_base,index=['Код_профессии_при_наличии'],
                                 columns=['прошли_ускоренное_обучение_по_индивидуаль-ным_учебным_планам'],
                                 values=['for_counting'],
                                 aggfunc='sum')
df_po_2_1_2_ind_up.columns = df_po_2_1_2_ind_up.columns.droplevel()

df_po_2_1_2 = pd.concat([df_po_2_1_2,df_po_2_1_2_ind_up],axis=1)

# Создаем проверку
df_po_2_1_2['temp'] = df_po_2_1_2['Всего по источникам финансирования'] == df_po_2_1_2['Всего по форме обучения']

df_po_2_1_2['Совпадение сумм столбцов 6-9 и 13-15'] = df_po_2_1_2['temp'].apply(lambda x:'СОВПАДАЕТ' if x is True else 'НЕ СОВПАДАЕТ!!!')
df_po_2_1_2.drop(columns='temp',inplace=True)
df_po_2_1_2.index.name = 'Код профессии при наличии'

In [1819]:
#Записываем датафрейм на лист
wb['Раздел 2.1.2'][f'A1'] = 'Распределение слушателей,обученных по программам переподготовки рабочих,служащих'
wb['Раздел 2.1.2'][f'A1'].font = font_name_table

for r in dataframe_to_rows(df_po_2_1_2,index =True,header=True):
    if len(r) != 1:
        wb['Раздел 2.1.2'].append(r)
wb['Раздел 2.1.2'][f'A2'] = 'В таблице отображаются ТОЛЬКО ТЕ показатели которые присутствуют в ИСХОДНОЙ таблице!!!!'
wb['Раздел 2.1.2'][f'A2'].font = font_name_table
wb['Раздел 2.1.2']['A2'].alignment = Alignment(wrap_text=True)

wb['Раздел 2.1.2'][f'A3'] = 'Код профессии при наличии'

wb['Раздел 2.1.2'].column_dimensions['A'].width = 50
wb['Раздел 2.1.2'].column_dimensions['C'].width = 15
wb['Раздел 2.1.2'].column_dimensions['D'].width = 15





wb['Раздел 2.1.2']['B2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.2']['C2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.2']['D2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.2']['E2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.2']['F2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.2']['G2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.2']['H2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.2']['I2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.2']['J2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.2']['K2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.2']['L2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.2']['M2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.2']['Q2'].alignment = Alignment(wrap_text=True)

In [1820]:
# Считаем лист 2.1.3
# Считаем раздел 2.1.2
df_po_2_1_3_base = df_po.copy()
df_po_2_1_3_base['for_counting'] = 1

# Отбираем слушателей обученных по программам переподготовки рабочих, служащих
df_po_2_1_3_base = df_po_2_1_3_base[df_po_2_1_3_base['Программа_профессионального_обучения_направление_подготовки'] == 'Программа повышения квалификации рабочих, служащих']

# Создаем сводную таблицу для колонок 6,7,8,9
df_po_2_1_3_6_9 = pd.pivot_table(df_po_2_1_3_base,index=['Код_профессии_при_наличии'],
                                columns=['Источник_финансирования_обучения'],
                                values=['for_counting'],
                                aggfunc='sum')
df_po_2_1_3_6_9.columns = df_po_2_1_3_6_9.columns.droplevel()

df_po_2_1_3_6_9['Всего по источникам финансирования'] = df_po_2_1_3_6_9.sum(axis=1)

#Создаем сводную таблицу для колонок 13,14,15
df_po_2_1_3_13_15 = pd.pivot_table(df_po_2_1_3_base,index=['Код_профессии_при_наличии'],
                                columns=['Форма_обучения'],
                                values=['for_counting'],
                                aggfunc='sum')
df_po_2_1_3_13_15.columns = df_po_2_1_3_13_15.columns.droplevel()

df_po_2_1_3_13_15['Всего по форме обучения'] = df_po_2_1_3_13_15.sum(axis=1)

df_po_2_1_3 = pd.concat([df_po_2_1_3_6_9,df_po_2_1_3_13_15],axis=1)

In [1821]:
# Считаем по категориям финансирования
df_po_2_1_3_cat_finance = pd.pivot_table(df_po_2_1_3_base,index=['Код_профессии_при_наличии'],
                                        columns=['Источник_финансирования_индикаторы_физ_лицо_юр_лицо_бюдж_ассигнования_собственные_средства_ЦОПП'],
                                        values=['for_counting'],
                                        aggfunc='sum')
df_po_2_1_3_cat_finance.columns = df_po_2_1_3_cat_finance.columns.droplevel()

df_po_2_1_3 = pd.concat([df_po_2_1_3,df_po_2_1_3_cat_finance],axis=1)




#Считаем женщин

df_po_2_1_3_women= pd.pivot_table(df_po_2_1_3_base,index=['Код_профессии_при_наличии'],
                                 columns=['Пол_получателя'],
                                 values=['for_counting'],
                                 aggfunc='sum')
df_po_2_1_3_women.columns = df_po_2_1_3_women.columns.droplevel()

df_po_2_1_3 = pd.concat([df_po_2_1_3,df_po_2_1_3_women],axis=1)



# Считаем тех кто прошел по индвидуальным учебным планам


df_po_2_1_3_ind_up = pd.pivot_table(df_po_2_1_3_base,index=['Код_профессии_при_наличии'],
                                 columns=['прошли_ускоренное_обучение_по_индивидуаль-ным_учебным_планам'],
                                 values=['for_counting'],
                                 aggfunc='sum')
df_po_2_1_3_ind_up.columns = df_po_2_1_3_ind_up.columns.droplevel()

df_po_2_1_3 = pd.concat([df_po_2_1_3,df_po_2_1_3_ind_up],axis=1)

# Создаем проверку
df_po_2_1_3['temp'] = df_po_2_1_3['Всего по источникам финансирования'] == df_po_2_1_3['Всего по форме обучения']

df_po_2_1_3['Совпадение сумм столбцов 6-9 и 13-15'] = df_po_2_1_3['temp'].apply(lambda x:'СОВПАДАЕТ' if x is True else 'НЕ СОВПАДАЕТ!!!')
df_po_2_1_3.drop(columns='temp',inplace=True)
df_po_2_1_3.index.name = 'Код профессии при наличии'

In [1822]:
#Записываем датафрейм на лист
wb['Раздел 2.1.3'][f'A1'] = 'Распределение слушателей,обученных по программам повышения квалификации рабочих,служащих'
wb['Раздел 2.1.3'][f'A1'].font = font_name_table

for r in dataframe_to_rows(df_po_2_1_3,index =True,header=True):
    if len(r) != 1:
        wb['Раздел 2.1.3'].append(r)
wb['Раздел 2.1.3'][f'A2'] = 'В таблице отображаются ТОЛЬКО ТЕ показатели которые присутствуют в ИСХОДНОЙ таблице!!!!'
wb['Раздел 2.1.3'][f'A2'].font = font_name_table
wb['Раздел 2.1.3']['A2'].alignment = Alignment(wrap_text=True)

wb['Раздел 2.1.3'][f'A3'] = 'Код профессии при наличии'
wb['Раздел 2.1.3'].column_dimensions['A'].width = 50
wb['Раздел 2.1.3'].column_dimensions['C'].width = 15
wb['Раздел 2.1.3'].column_dimensions['D'].width = 15





wb['Раздел 2.1.3']['B2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.3']['C2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.3']['D2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.3']['E2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.3']['F2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.3']['G2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.3']['H2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.3']['I2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.3']['J2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.3']['K2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.3']['L2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.3']['M2'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.1.3']['Q2'].alignment = Alignment(wrap_text=True)

In [1848]:
#Считаем лист 2.2
df_po_2_2_base = df_po.copy()
df_po_2_2_base['for_counting'] = 1

In [ ]:
#Создаес сводную таблицу


In [1823]:
#Считаем лист 2.4
df_po_2_4 = df_po.copy()
df_po_2_4['for_counting'] = 1

In [1824]:
# Считаем общее количество
df_po_2_4_all = pd.pivot_table(df_po_2_4,columns=['Программа_профессионального_обучения_направление_подготовки'],
                              values=['for_counting'],
                              aggfunc='sum')
df_po_2_4_all.index = ['Всего обучено']

In [1825]:
df_po_2_4_all

Программа_профессионального_обучения_направление_подготовки,"Программа переподготовки рабочих, служащих","Программа повышения квалификации рабочих, служащих","Программа профессиональной подготовки по профессии рабочего, должности служащего"
Всего обучено,6,2,4


In [1826]:
# Считаем по основному общему образованию
df_po_2_4_02 = df_po_2_4[df_po_2_4['Уровень_образования']=='основное общее (9 классов)']


In [1827]:
df_po_2_4_02

,Порядковый_номер_группы,Вид_документа,Статус_документа,Подтверждение_утраты,Подтверждение_обмена,Подтверждение_уничтожения,Серия_документа,Номер_документа,Дата_выдачи_документа,Регистрационный_номер,...,Отметка_об_оплате_оплачено_не_оплачено_в_рамках_индикаторов,Использование_сетевой_формы_обучения,Использование_ЭО,Использование_ДОТ,прошли_ускоренное_обучение_по_индивидуаль-ным_учебным_планам,Номер_приказа_о_зачислении,Номер_приказа_об_отчислении,Текущий_возраст,Возрастная_категория_1ПО,for_counting


In [1828]:
# считаем сумму
df_po_2_4_02_all = pd.pivot_table(df_po_2_4_02,columns=['Программа_профессионального_обучения_направление_подготовки'],
                                 values=['for_counting'],
                                 aggfunc='sum')

In [1829]:
df_po_2_4_02['Дата_выдачи_оригинала'].dtype

dtype('<M8[ns]')

In [1830]:
#Считаем лист 2.5
# Создаем дополнительную числовую колонку где каждое значение это 1, для удобства агрегирования
df_po_2_5 = df_po.copy()
# Добавляем колонку с 1
df_po_2_5['for_counting'] = 1 


In [1831]:
# Считаем строку 01 Всего
df_2_5_all = pd.pivot_table(df_po_2_5,              
              columns=['Программа_профессионального_обучения_направление_подготовки','Источник_финансирования_обучения'],
              values=['for_counting'],
              aggfunc='sum')
df_2_5_all.index = ['Всего']
df_2_5_all.index.name='Код государства по ОКСМ'

In [1832]:
df_2_5_all

Программа_профессионального_обучения_направление_подготовки Программа переподготовки рабочих, служащих  \
Источник_финансирования_обучения                                                        Местный бюджет   
Код государства по ОКСМ                                                                                  
Всего                                                                                                1   

Программа_профессионального_обучения_направление_подготовки                   \
Источник_финансирования_обучения                            Платное обучение   
Код государства по ОКСМ                                                        
Всего                                                                      1   

Программа_профессионального_обучения_направление_подготовки                      \
Источник_финансирования_обучения                            Региональный бюджет   
Код государства по ОКСМ                                                           
Всего                                                                         1   

Программа_профессионального_обучения_направление_подготовки                     \
Источник_финансирования_обучения                            Федеральный бюджет   
Код государства по ОКСМ                                                          
Всего                                                                        3   

Программа_профессионального_обучения_направление_подготовки Программа повышения квалификации рабочих, служащих  \
Источник_финансирования_обучения                                                                Местный бюджет   
Код государства по ОКСМ                                                                                          
Всего                                                                                                        1   

Программа_профессионального_обучения_направление_подготовки                      \
Источник_финансирования_обучения                            Региональный бюджет   
Код государства по ОКСМ                                                           
Всего                                                                         1   

Программа_профессионального_обучения_направление_подготовки Программа профессиональной подготовки по профессии рабочего, должности служащего  \
Источник_финансирования_обучения                                                                                              Местный бюджет   
Код государства по ОКСМ                                                                                                                        
Всего                                                                                                        1                                 

Программа_профессионального_обучения_направление_подготовки                   \
Источник_финансирования_обучения                            Платное обучение   
Код государства по ОКСМ                                                        
Всего                                                                      2   

Программа_профессионального_обучения_направление_подготовки                     
Источник_финансирования_обучения                            Федеральный бюджет  
Код государства по ОКСМ                                                         
Всего                                                                        1

In [1833]:
# Считаем данные по странам
svod_df_po_2_5=pd.pivot_table(df_po_2_5,
              index=['Гражданство_получателя_код_страны_по_ОКСМ'],
              columns=['Программа_профессионального_обучения_направление_подготовки','Источник_финансирования_обучения'],
              values=['for_counting'],
              aggfunc='sum')

In [1834]:
# Удаляем лишний мультииндекс
svod_df_po_2_5.columns= svod_df_po_2_5.columns.droplevel()
# заполняем нулями для корректного суммирования
svod_df_po_2_5.fillna(0.0,inplace=True)


In [1835]:
# Соединяем датафреймы
df_po_2_5_out = pd.concat([df_2_5_all,svod_df_po_2_5])
# заменяем нули на нан, чтобы в итоговой таблице нули не отвлекали
df_po_2_5_out.replace(0.0,np.NaN,inplace=True)

In [1836]:
df_po_2_5_out

Программа_профессионального_обучения_направление_подготовки Программа переподготовки рабочих, служащих  \
Источник_финансирования_обучения                                                        Местный бюджет   
Всего                                                                                              1.0   
12                                                                                                 NaN   
20                                                                                                 NaN   
4                                                                                                  NaN   
643                                                                                                1.0   
644                                                                                                NaN   

Программа_профессионального_обучения_направление_подготовки                   \
Источник_финансирования_обучения                            Платное обучение   
Всего                                                                    1.0   
12                                                                       NaN   
20                                                                       NaN   
4                                                                        NaN   
643                                                                      1.0   
644                                                                      NaN   

Программа_профессионального_обучения_направление_подготовки                      \
Источник_финансирования_обучения                            Региональный бюджет   
Всего                                                                       1.0   
12                                                                          NaN   
20                                                                          NaN   
4                                                                           NaN   
643                                                                         1.0   
644                                                                         NaN   

Программа_профессионального_обучения_направление_подготовки                     \
Источник_финансирования_обучения                            Федеральный бюджет   
Всего                                                                      3.0   
12                                                                         NaN   
20                                                                         NaN   
4                                                                          NaN   
643                                                                        3.0   
644                                                                        NaN   

Программа_профессионального_обучения_направление_подготовки Программа повышения квалификации рабочих, служащих  \
Источник_финансирования_обучения                                                                Местный бюджет   
Всего                                                                                                      1.0   
12                                                                                                         NaN   
20                                                                                                         NaN   
4                                                                                                          NaN   
643                                                                                                        NaN   
644                                                                                                        1.0   

Программа_профессионального_обучения_направление_подготовки                      \
Источник_финансирования_обучения                            Региональный бюджет   
Всего                                                                       1.0   
12                        

In [1837]:
wb['Раздел 2.5'][f'A1'] = 'Распределение слушателей, обученных по программам ПО, по гражданству'
wb['Раздел 2.5'][f'A1'].font = font_name_table
wb['Раздел 2.5'][f'A2'] = 'В таблице отображаются ТОЛЬКО ТЕ показатели которые присутствуют в ИСХОДНОЙ таблице!!!'
wb['Раздел 2.5'][f'A2'].font = font_name_table



for r in dataframe_to_rows(df_po_2_5_out,index =True,header=True):
    if len(r) != 1:
        wb['Раздел 2.5'].append(r)
wb['Раздел 2.5'].column_dimensions['A'].width = 50
wb['Раздел 2.5'].column_dimensions['B'].width = 50
wb['Раздел 2.5'].column_dimensions['F'].width = 50

In [1838]:
# Создаем раздел 2.6
df_po_2_6 = df_po.copy()
df_po_2_6['for_counting'] = 1

In [1839]:
# Считаем в общем сколько обучено
df_po_2_6_all = pd.pivot_table(df_po_2_6,columns=['Программа_профессионального_обучения_направление_подготовки','Пол_получателя'],
              values=['for_counting'],
              aggfunc='sum')
df_po_2_6_all.index = ['Всего обучено']

In [1840]:
df_po_2_6_all

Программа_профессионального_обучения_направление_подготовки Программа переподготовки рабочих, служащих  \
Пол_получателя                                                                                     Жен   
Всего обучено                                                                                        2   

Программа_профессионального_обучения_направление_подготовки      \
Пол_получателя                                              Муж   
Всего обучено                                                 4   

Программа_профессионального_обучения_направление_подготовки Программа повышения квалификации рабочих, служащих  \
Пол_получателя                                                                                             Жен   
Всего обучено                                                                                                1   

Программа_профессионального_обучения_направление_подготовки      \
Пол_получателя                                              Муж   
Всего обучено                                                 1   

Программа_профессионального_обучения_направление_подготовки Программа профессиональной подготовки по профессии рабочего, должности служащего  \
Пол_получателя                                                                                                                           Жен   
Всего обучено                                                                                                3                                 

Программа_профессионального_обучения_направление_подготовки      
Пол_получателя                                              Муж  
Всего обучено                                                 1

In [1841]:
# Считаем распределение по возрастам
df_po_2_6_by_age = pd.pivot_table(df_po_2_6,index=['Возрастная_категория_1ПО'],
                                 columns=['Программа_профессионального_обучения_направление_подготовки','Пол_получателя'],
                                 values=['for_counting'],
                                 aggfunc='sum')

In [1842]:
df_po_2_6_by_age

for_counting  \
Программа_профессионального_обучения_направление_подготовки Программа переподготовки рабочих, служащих   
Пол_получателя                                                                                     Жен   
Возрастная_категория_1ПО                                                                                 
16 лет                                                                                             NaN   
17 лет                                                                                             1.0   
18 лет                                                                                             1.0   
19 лет                                                                                             NaN   
21 год                                                                                             NaN   
45-49 лет                                                                                          NaN   
65 лет и старше                                                                                    NaN   
В возрасте моложе 14 лет                                                                           NaN   

                                                                  \
Программа_профессионального_обучения_направление_подготовки        
Пол_получателя                                               Муж   
Возрастная_категория_1ПО                                           
16 лет                                                       1.0   
17 лет                                                       NaN   
18 лет                                                       NaN   
19 лет                                                       1.0   
21 год                                                       1.0   
45-49 лет                                                    NaN   
65 лет и старше                                              NaN   
В возрасте моложе 14 лет                                     1.0   

                                                                                                                \
Программа_профессионального_обучения_направление_подготовки Программа повышения квалификации рабочих, служащих   
Пол_получателя                                                                                             Жен   
Возрастная_категория_1ПО                                                                                         
16 лет                                                                                                     NaN   
17 лет                                                                                                     NaN   
18 лет                                                                                                     NaN   
19 лет                                                                                                     NaN   
21 год                                                                                                     NaN   
45-49 лет                                                                                                  1.0   
65 лет и старше                                                                                            NaN   
В возрасте моложе 14 лет                                                                                   NaN   

                                                                  \
Программа_профессионального_обучения_направление_подготовки        
Пол_получателя                                               Муж   
Возрастная_категория_1ПО                                           
16 лет                                                       NaN   
17 лет                                                       NaN   
18 лет                                                       NaN   
19 лет                                                       NaN   
21 год                                                       1.0   
45-49 лет          

In [1843]:
# Удаляем лишний мультииндекс
df_po_2_6_by_age.columns= df_po_2_6_by_age.columns.droplevel()

In [1844]:
# Соединяем 2 датафрейма
df_po_2_6_out = pd.concat([df_po_2_6_all,df_po_2_6_by_age])

In [1845]:
df_po_2_6_out

Программа_профессионального_обучения_направление_подготовки Программа переподготовки рабочих, служащих  \
Пол_получателя                                                                                     Жен   
Всего обучено                                                                                      2.0   
16 лет                                                                                             NaN   
17 лет                                                                                             1.0   
18 лет                                                                                             1.0   
19 лет                                                                                             NaN   
21 год                                                                                             NaN   
45-49 лет                                                                                          NaN   
65 лет и старше                                                                                    NaN   
В возрасте моложе 14 лет                                                                           NaN   

Программа_профессионального_обучения_направление_подготовки       \
Пол_получателя                                               Муж   
Всего обучено                                                4.0   
16 лет                                                       1.0   
17 лет                                                       NaN   
18 лет                                                       NaN   
19 лет                                                       1.0   
21 год                                                       1.0   
45-49 лет                                                    NaN   
65 лет и старше                                              NaN   
В возрасте моложе 14 лет                                     1.0   

Программа_профессионального_обучения_направление_подготовки Программа повышения квалификации рабочих, служащих  \
Пол_получателя                                                                                             Жен   
Всего обучено                                                                                              1.0   
16 лет                                                                                                     NaN   
17 лет                                                                                                     NaN   
18 лет                                                                                                     NaN   
19 лет                                                                                                     NaN   
21 год                                                                                                     NaN   
45-49 лет                                                                                                  1.0   
65 лет и старше                                                                                            NaN   
В возрасте моложе 14 лет                                                                                   NaN   

Программа_профессионального_обучения_направление_подготовки       \
Пол_получателя                                               Муж   
Всего обучено                                                1.0   
16 лет                                                       NaN   
17 лет                                                       NaN   
18 лет                                                       NaN   
19 лет                                                       NaN   
21 год                                                       1.0   
45-49 лет                                                    NaN   
65 лет и старше                                              NaN   
В возрасте моложе 14 лет                                     NaN   

Программа_профессионального_обучения_направление_подготовки Программа профессион

In [1846]:
wb['Раздел 2.6'][f'A1'] = 'Распределение слушателей, обученных по программам ПО, по возрасту и полу'
wb['Раздел 2.6'][f'A1'].font = font_name_table
wb['Раздел 2.6'][f'A2'] = 'В таблице отображаются ТОЛЬКО ТЕ показатели которые присутствуют в ИСХОДНОЙ таблице!!!'
wb['Раздел 2.6'][f'A2'].font = font_name_table
wb['Раздел 2.6'][f'A3'] = 'Чтобы вычислить показатель ВСЕГО ОБУЧЕНО сложите значения в колонке Муж+Жен'
wb['Раздел 2.6'][f'A3'].font = font_name_table



for r in dataframe_to_rows(df_po_2_6_out,index =True,header=True):
    if len(r) != 1:
        wb['Раздел 2.6'].append(r)
wb['Раздел 2.6'].column_dimensions['A'].width = 50
wb['Раздел 2.6'].column_dimensions['B'].width = 50
wb['Раздел 2.6']['B4'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.6'].column_dimensions['D'].width = 50
wb['Раздел 2.6']['D4'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.6'].column_dimensions['F'].width = 50
wb['Раздел 2.6']['F4'].alignment = Alignment(wrap_text=True)



In [1847]:
 # Получаем текущее время для того чтобы использовать в названии
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
# Сохраняем итоговый файл
wb.save(f'{path_to_end_folder_report}/Часть отчета 1-ПО.xlsx {current_time}.xlsx')